In [1]:
import time
import random
import json
import logging
from datetime import datetime
import requests
import yaml

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load configuration
def load_config():
    try:
        with open('config.yaml', 'r') as config_file:
            return yaml.safe_load(config_file)
    except FileNotFoundError:
        logging.error("Configuration file 'config.yaml' not found.")
        return None
    except yaml.YAMLError as e:
        logging.error(f"Error parsing configuration file: {e}")
        return None

# Simulate IoT sensor data for a motor
def get_sensor_data(config):
    temperature = round(random.uniform(config['temp_min'], config['temp_max']), 2)
    vibration = round(random.uniform(config['vibration_min'], config['vibration_max']), 2)
    rpm = round(random.uniform(config['rpm_min'], config['rpm_max']), 0)
    power = round(random.uniform(config['power_min'], config['power_max']), 2)

    return {
        "timestamp": datetime.now().isoformat(),
        "temperature": temperature,
        "vibration": vibration,
        "rpm": rpm,
        "power": power
    }

# Function to send sensor data to cloud API
def send_data_to_cloud(data, config):
    try:
        response = requests.post(config['cloud_api_url'], json=data, timeout=5)
        response.raise_for_status()
        logging.info(f"Data sent successfully: {data}")
        return True
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to send data: {e}")
        return False

# Save data locally if cloud upload fails
def save_data_locally(data, config):
    try:
        with open(config['local_data_file'], 'a') as f:
            json.dump(data, f)
            f.write('\n')
        logging.info(f"Data saved locally: {data}")
    except IOError as e:
        logging.error(f"Failed to save data locally: {e}")

# Continuously collect and send sensor data
def run_simulation(config):
    while True:
        try:
            sensor_data = get_sensor_data(config)
            if not send_data_to_cloud(sensor_data, config):
                save_data_locally(sensor_data, config)
            time.sleep(config['collection_interval'])
        except KeyboardInterrupt:
            logging.info("Simulation stopped by user.")
            break
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            time.sleep(config['error_retry_interval'])

if __name__ == "__main__":
    config = load_config()
    if config:
        logging.info("Starting IoT data collection simulation...")
        run_simulation(config)
    else:
        logging.error("Failed to load configuration. Exiting.")

ERROR:root:Configuration file 'config.yaml' not found.
ERROR:root:Failed to load configuration. Exiting.
